In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
import sys
sys.path.append(r'C:\\Users\\scanimage\\Documents\\JJM\\post_cnmfe_analysis')
import loading_utils_jjm as utils_jjm
import loading_utils_jjm as utils_jjm

C:\Users\scanimage\AppData\Local\Temp\ipykernel_49520\3136755228.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [6]:
CNMFE_session = '25-Mar_13_27_27_out.mat'
pathToCNMFE = r'F:\\JJM\\\25-Mar_13_27_27_out.mat'

In [7]:
## load cnmfe output and perform some basic adjustments (normalize traces to peak intensity, calculate z score)
cell_fluorescence = sio.loadmat(pathToCNMFE)
C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
spatial_components=np.array(cell_fluorescence['A'].todense())

In [9]:
com_df, spatial_components = utils_jjm.return_spatial_info(pathToCNMFE, 0.6, dims=(752, 480))
cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components, dims=(752, 480))

In [10]:
grouped_raw_data = {}
grouped_raw_data[CNMFE_session] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 
                               'C_normalized_z_scored': C_normalized_z_scored, 'com' : com_df, 
                               'spatial_components' : spatial_components, 
                               'cell_contours': cell_contours, 'for_dims' : for_dims}

In [11]:
##remove incorrectly identified cells
cells_to_drop = []
C_traces_filtered_for_analysis = {}
com_filtered_for_analysis = {}
for session in list(grouped_raw_data.keys()):
        # get indicies of small cells 
        cells_to_drop = np.array([int(cell) for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
        C_traces_filtered = grouped_raw_data[session]['C_normalized_z_scored'].drop(cells_to_drop, axis=1)
    
        C_traces_filtered_for_analysis[session] = C_traces_filtered.drop('msCamFrame', axis=1)
        com_filtered_for_analysis[session] = grouped_raw_data[session]['com'].drop(cells_to_drop, axis=0).transpose()
print('cell count:')

cell count:


In [ ]:
# sets sampling interval to 5Hz to correspond to behavior data 
new_sampling_interval = .2
C_z_score_df = pd.concat([C_traces_filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(C_traces_filtered_for_analysis.keys())], axis=1, keys=list(C_traces_filtered_for_analysis.keys()))

In [ ]:
# after downsampling convert back to regular index, timedelta is behaving oddly 
C_z_score_df_reindexed = pd.concat([C_z_score_df[session].reset_index(drop=True) 
                for session in list(set(C_z_score_df.columns.get_level_values(0)))], axis=1, keys=list(set(C_z_score_df.columns.get_level_values(0))))

In [ ]:
# drop columns with all nan values
C_z_score_df_reindexed.dropna(axis=1, how='all', inplace=True)